# Few shot examples for chat models

This notebook covers how to use few shot examples in chat models. There does not appear to be solid consensus on how best to do few shot prompting, and the optimal prompt compilation will likely vary by model. Because of this, we provide few-shot prompt templates like the [FewShotChatMessagePromptTemplate](https://api.python.langchain.com/en/latest/prompts/langchain.prompts.few_shot.FewShotChatMessagePromptTemplate.html) as a flexible starting point, and you can modify or replace them as you see fit.

The goal of few-shot prompt templates are to dynamically select examples based on an input, and then format the examples in a final prompt to provide for the model.


**Note:** The following code examples are for chat models. For similar few-shot prompt examples for completion models (LLMs), see the [few-shot prompt templates](few_shot_examples) guide.

### Fixed Examples

The most basic (and common) few-shot prompting technique is to use a fixed prompt example. This way you can select a chain, evaluate it, and avoid worrying about additional moving parts in production.

The basic components of the template are:
- `examples`: A list of dictionary examples to include in the final prompt.
- `prefix`: message(s) or template(s) include at the beginning. Usually a system message instructing the model to behave in a certain fashion, providing the time of day, injecting a user's custom instructions, or providing other information.
- `example_prompt`: converts each example into 1 or more messages through its [`format_messages`](https://api.python.langchain.com/en/latest/prompts/langchain.prompts.chat.ChatPromptTemplate.html#langchain.prompts.chat.ChatPromptTemplate.format_messages) method. A common example would be to convert each example into one human message and one AI message response, or a human message followed by a function call message.
- `suffix`: Additional message(s) or template(s) to insert after the few-shot examples. This is typically a `HumanMessagePromptTemplate` where the chain's input will be inserted.

Below is a simple demonstration. First, import the modules for this example:

In [1]:
from langchain.schema import SystemMessage
from langchain.prompts import (
    ChatPromptTemplate,
    FewShotChatMessagePromptTemplate,
    HumanMessagePromptTemplate,
    AIMessagePromptTemplate,
)

Then, define the examples you'd like to include.

In [2]:
examples = [
    {"input": "2+2", "output": "4"},
    {"input": "2+3", "output": "5"},
]

Finally, assemble them into the few-shot prompt template.

In [3]:
# This is a prompt template used to format each individual example.
example_prompt = ChatPromptTemplate.from_messages(
    [
        HumanMessagePromptTemplate.from_template("{input}"),
        AIMessagePromptTemplate.from_template("{output}"),
    ]
)
few_shot_prompt = FewShotChatMessagePromptTemplate(
    input_variables=["user_input"],
    prefix=[SystemMessage(content="You are a helpful AI Assistant")],
    example_prompt=example_prompt,
    examples=examples,
    suffix=[HumanMessagePromptTemplate.from_template("{user_input}")],
)
print(few_shot_prompt.format(user_input="What is 4+4?"))

System: You are a helpful AI Assistant
Human: 2+2
AI: 4
Human: 2+3
AI: 5
Human: What is 4+4?


#### Use within an LLMChain

You can directly use this in an LLMChain. Here is an example:

In [4]:
# Use within an LLMChain
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI

chain = LLMChain(
    llm=ChatOpenAI(),
    prompt=few_shot_prompt,
)
chain.predict(user_input="What's 3+3?")

'3+3 equals 6.'

## Dynamic Few-shot Prompting

Sometimes you may want to condition which examples are shown based on the input. For this, you can replace the `examples` with an `example_selector`. The other components remain the same as above! To review, the dynamic few-shot prompt template would look like:

- `example_selector`: responsible for selecting few-shot examples (and the order in which they are returned) for a given input. These implement the [BaseExampleSelector](https://api.python.langchain.com/en/latest/prompts/langchain.prompts.example_selector.base.BaseExampleSelector.html#langchain.prompts.example_selector.base.BaseExampleSelector) interface. A common example is the vectorstore-backed [SemanticSimilarityExampleSelector](https://api.python.langchain.com/en/latest/prompts/langchain.prompts.example_selector.semantic_similarity.SemanticSimilarityExampleSelector.html#langchain.prompts.example_selector.semantic_similarity.SemanticSimilarityExampleSelector)
- `prefix`: message(s) or template(s) to prepend to the final prompt. Usually a system message instructing the model to behave in a certain fashion, providing the time of day, injecting a user's custom instructions, or providing other information.
- `example_prompt`: convert each example into 1 or more messages through its [`format_messages`](https://api.python.langchain.com/en/latest/prompts/langchain.prompts.chat.ChatPromptTemplate.html#langchain.prompts.chat.ChatPromptTemplate.format_messages) method. A common example would be to convert each example into one human message and one AI message response, or a human message followed by a function call message.
- `suffix`: Additional message(s) or template(s) to insert after the few-shot examples. This is typically a `HumanMessagePromptTemplate` where the chain's input will be inserted.

In [5]:
from langchain.prompts import SemanticSimilarityExampleSelector
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

Since we are using a vectorstore to select examples based on semantic similarity, we will want to first populate the store.

In [6]:
examples = [
    {"input": "2+2", "output": "4"},
    {"input": "2+3", "output": "5"},
    {"input": "2+4", "output": "6"},
    {"input": "What did the cow say to the moon?", "output": "nothing at all"},
    {
        "input": "Write me a poem about the moon",
        "output": "One for the moon, and one for me, who are we to talk about the moon?",
    },
]

to_vectorize = [" ".join(example.values()) for example in examples]
embeddings = OpenAIEmbeddings()
vectorstore = Chroma.from_texts(to_vectorize, embeddings, metadatas=examples)

#### Create the `example_selector`

With a vectorstore created, you can create the `example_selector`. Here we will isntruct it to only fetch the top 2 examples.

In [7]:
example_selector = SemanticSimilarityExampleSelector(
    vectorstore=vectorstore,
    k=2,
)

# The prompt template will load examples by passing the input do the `select_examples` method
example_selector.select_examples({"input": "horse"})

[{'input': 'What did the cow say to the moon?', 'output': 'nothing at all'},
 {'input': '2+4', 'output': '6'}]

#### Create prompt template

Assemble the prompt template, using the `example_selector` created above.

In [8]:
from langchain.schema import SystemMessage
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate

# Define how each example will be formatted.
# In this case, each example will become 2 messages:
# 1 human, and 1 AI
example_prompt = ChatPromptTemplate.from_role_strings(
    [("user", "{input}"), ("assistant", "{output}")]
)

# Define the overall prompt.
few_shot_prompt = FewShotChatMessagePromptTemplate(
    input_variables=[
        "input"
    ],  # The input variable checks that the correct key is provided by the chain
    prefix=[SystemMessage(content="You are a helpful AI Assistant")],
    example_selector=example_selector,
    example_prompt=example_prompt,
    suffix=[HumanMessagePromptTemplate.from_template("{input}")],
)

Below is an example of how this would be assembled.

In [9]:
print(few_shot_prompt.format(input="What's 3+3?"))

System: You are a helpful AI Assistant
user: 2+3
assistant: 5
user: 2+2
assistant: 4
Human: What's 3+3?


#### Use the prompt in an LLMChain

Now it's time to use the prompt in a chain! 

In [10]:
# Use within an LLMChain
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI

chain = LLMChain(
    llm=ChatOpenAI(),
    prompt=few_shot_prompt,
)
chain.predict(input="What's 3+3?")

'3 + 3 is equal to 6.'